In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='3'
import tensorflow as tf
import random
import time
import datetime
# from test import test
from dataloader_yun import DataManager
# from generator import Generator
# from discriminator import Discriminator
# from rollout import ROLLOUT
import numpy as np
# from discriminator import Discriminator
import re

In [2]:
# data_dir = '/home/yunzhao/PJ/seqGan/AC_SEQ_GAN/PCNN_GAN/Discriminator/data'
data_dir = '/home/rohanjain/data/sentgen/'
word2vec_file = os.path.join(data_dir, 'vec.txt')
relation2id_file = os.path.join(data_dir, 'RE/relation2id.txt')
datamanager = DataManager(100, word2vec_file, relation2id_file)
entity2id_file = os.path.join(data_dir, 'RE/entity2id.txt')
training_data = datamanager.load_training_data(entity2id_file,filename=os.path.join(data_dir, 'RE/train.txt'))
training_data = np.array(training_data)
testing_data = datamanager.load_testing_data(filename=os.path.join(data_dir, 'RE/test.txt'))
print(str(len(training_data))+" "+str(len(testing_data)))


RelationTotal: 53
WordTotal:  114043
Word dimension:  50
Start loading training data.
Start loading testing data.
570088 96678


In [4]:
sample=training_data[0]
sample

Sentence(entity1=queens| entity2=belle_harbor| relation=Relation(id=48| name=/location/location/contains| number=0)| words=sen. charles e. schumer called on federal safety officials yesterday to reopen their investigation into the fatal crash of a passenger jet in belle_harbor , queens , because equipment failure , not pilot error , might have been the cause .)

In [6]:
sample.words

['sen.',
 'charles',
 'e.',
 'schumer',
 'called',
 'on',
 'federal',
 'safety',
 'officials',
 'yesterday',
 'to',
 'reopen',
 'their',
 'investigation',
 'into',
 'the',
 'fatal',
 'crash',
 'of',
 'a',
 'passenger',
 'jet',
 'in',
 'belle_harbor',
 ',',
 'queens',
 ',',
 'because',
 'equipment',
 'failure',
 ',',
 'not',
 'pilot',
 'error',
 ',',
 'might',
 'have',
 'been',
 'the',
 'cause',
 '.']

In [5]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    #string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"[^A-Za-z_]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r",", "", string)
    return string.strip().lower()

In [7]:
for sample in training_data:
    sample.words=clean_str(' '.join(sample.words))

In [8]:
sample=training_data[0]
sample.words
#sample.words.split()

'sen charles e schumer called on federal safety officials yesterday to reopen their investigation into the fatal crash of a passenger jet in belle_harbor queens because equipment failure not pilot error might have been the cause'

In [14]:
sample.relation.id

48

In [10]:
sample.entity1

'queens'

In [11]:
sample.entity2

'belle_harbor'

In [12]:
def get_entity_loc(d):
    e1 = d.entity1
    e2 = d.entity2
    words = d.words.split()
    l1 = 0
    l2 = 0
    for i, w in enumerate(words):
        if w == e1:
            l1 = i
        if w == e2:
            l2 = i
    return (l1,l2)

In [13]:
get_entity_loc(sample)

(24, 23)

In [15]:
all_words = []
type(all_words)
for sample in training_data:
    all_words.extend(sample.words.split())

In [19]:
len(all_words)

19665121

In [18]:
len(set(all_words))

179079

In [20]:
from collections import Counter
counter1 = Counter(all_words)

In [21]:
top_5020_words = counter1.most_common(5020)

In [22]:
top_5020_words[:20]

[('the', 1099982),
 ('and', 711880),
 ('of', 639166),
 ('in', 587666),
 ('a', 463413),
 ('to', 409577),
 ('s', 240625),
 ('for', 158385),
 ('that', 155412),
 ('on', 150734),
 ('lrb', 142085),
 ('rrb', 142059),
 ('by', 136366),
 ('with', 135657),
 ('at', 135480),
 ('is', 122569),
 ('from', 116117),
 ('as', 107535),
 ('his', 105924),
 ('was', 101924)]

In [29]:
words_tobe_considered = top_5020_words[12:]

In [30]:
vocab = [x[0] for x in words_tobe_considered]
type(vocab)

list

In [31]:
vocab

['by',
 'with',
 'at',
 'is',
 'from',
 'as',
 'his',
 'was',
 'he',
 'who',
 'said',
 'an',
 'has',
 'mr',
 'it',
 'new',
 'have',
 'are',
 'be',
 'had',
 'but',
 'its',
 'which',
 'not',
 'this',
 'about',
 'her',
 'one',
 'will',
 'two',
 'american',
 'president',
 'like',
 'n',
 'their',
 'after',
 'or',
 'were',
 'new_york',
 'china',
 'when',
 'last',
 'would',
 'year',
 'more',
 'they',
 'been',
 'iraq',
 'years',
 'other',
 'united',
 'i',
 'first',
 'up',
 'york',
 'also',
 'states',
 'she',
 'where',
 'm',
 'brooklyn',
 'than',
 'all',
 'over',
 'bush',
 'city',
 'former',
 'three',
 'out',
 'into',
 'state',
 'most',
 'united_states',
 'there',
 'including',
 'john',
 'wife',
 'manhattan',
 'world',
 'some',
 'now',
 'time',
 'company',
 'no',
 'washington',
 'between',
 'house',
 'against',
 'p',
 'through',
 'people',
 'j',
 'before',
 'war',
 'home',
 'we',
 'david',
 'th',
 'if',
 'him',
 'many',
 't',
 'what',
 'made',
 'only',
 'so',
 'could',
 'c',
 'well',
 'japan',


In [ ]:
file=open('vocab2id.txt','a')
i=0
file.write('UNK'+' '+str(i)+'\n')
for word in vocab:
    i+=1
    file.write(word+' '+str(i)+'\n')
file.close()

In [26]:
for i,sample in enumerate(training_data[:1000]):
    if 'lrb' in (sample.words.split(' ')):
        print(i, '----' , sample.words)
        print('___________')

11 ---- p m lrb mtv rrb the hills on laguna_beach the real orange county lauren_conrad lrb right rrb was the emotional blonde who wore every smidgen of excitement or frustration on her face sometimes both at once
___________
13 ---- p m lrb mtv rrb the hills on laguna_beach the real orange county lauren_conrad lrb right rrb was the emotional blonde who wore every smidgen of excitement or frustration on her face sometimes both at once
___________
23 ---- at the hotel on rivington on manhattan s lower_east_side the bathroom is not only visible from the bed but the floor to ceiling windows allow neighbors on delancey street to spy into the shower lrb though the hotel will install a plastic film on the window on request rrb
___________
26 ---- meanwhile these are our recommendations for the four civil court seats that are actually being contested in the upcoming democratic primary in manhattan and brooklyn manhattan second district lrb east village lower_east_side soho noho little italy an

In [28]:
training_data[495]

Sentence(entity1=jamie_carragher| entity2=liverpool| relation=Relation(id=36| name=/people/person/place_lived| number=0)| words=l i v e r p o o l   w h i c h   w o n   t h e   e u r o p e a n   c l u b   t i t l e   f o u r   o t h e r   t i m e s   l r b   a n d   r r b   w a s   c o a c h e d   b y   a   s p a n i a r d   l r b   r a f a e l   b e n i t e z   r r b   a n d   h a d   o n l y   t w o   e n g l i s h   p l a y e r s   o n   t h e   f i e l d   l r b   s t e v e n   g e r r a r d   a n d   j a m i e _ c a r r a g h e r   r r b)